In [4]:
import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("semcor") 
nltk.download("stopwords")
# nltk.download("averaged_perceptron_tagger")
nltk.download('omw-1.4')

from nltk import word_tokenize
from nltk.corpus import semcor 
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

import warnings
import numpy as np
from tqdm.notebook import tqdm
from string import punctuation
from num2words import num2words
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cheta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cheta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package semcor to
[nltk_data]     C:\Users\cheta\AppData\Roaming\nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cheta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cheta\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
semcor_tagged_sents = semcor.tagged_sents(tag='sem')
semcor_untagged_sents = semcor.sents()

In [6]:
import gensim.downloader as api

#online loading from api
from gensim.models import KeyedVectors
path = "./GoogleNews-vectors-negative300.bin"
word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

#local loading
# word2vec = api.load('word2vec-google-news-300')

In [7]:
EXTRA_SW = [
    "''",
    "'s",
    "``"
]

SW = stopwords.words("english")
SW += [p for p in punctuation]
SW += EXTRA_SW
# if '(' in SW:
#     print(1)

In [8]:
def sentence_vector(sent):
    sent_vec = np.zeros(300)
    for word in sent:
        if word in word2vec:
            sent_vec += word2vec.get_vector(word)
    return sent_vec  

In [9]:
senses = []
for i in range(len(semcor_tagged_sents)):
    for j in range(len(semcor_tagged_sents[i])):
        if isinstance(semcor_tagged_sents[i][j],nltk.Tree):
            try:
                if semcor_tagged_sents[i][j].height() == 3:
                    for tree in semcor_tagged_sents[i][j]:
                        if(tree.label() == 'NE'):
                            senses.append('NE')
                            
                else:
                    sense = semcor_tagged_sents[i][j].label().synset().name()
                    senses.append(sense)
                    
            except:
                if (semcor_tagged_sents[i][j].label()=='NE'):
                    senses.append('NE')
                else:
                    sense = semcor_tagged_sents[i][j].label()
                    senses.append(sense)
        else:
            senses.append('none')
senses.append('notag')
senses = list(set(senses))
senses = {senses[i]:i for i in range(len(senses))}

In [10]:
def get_most_frequent_sense_dict(tagged_sents,untagged_sents):
    sense_dict = {}
    for i in range(len(tagged_sents)):
        for j in range(len(tagged_sents[i])):
            if untagged_sents[i][j] not in sense_dict.keys():
                sense_dict[untagged_sents[i][j]] = []
            if isinstance(tagged_sents[i][j],nltk.Tree):
                try:
                    if tagged_sents[i][j].height() == 3:
                        for tree in tagged_sents[i][j]:
                            if(tree.label() == 'NE'):
                                sense_dict[untagged_sents[i][j]].append('NE')
                                
                    else:
                        sense = tagged_sents[i][j].label().synset().name()
                        sense_dict[untagged_sents[i][j]].append(sense)
                        
                except:
                    if (tagged_sents[i][j].label()=='NE'):
                        sense_dict[untagged_sents[i][j]].append('NE')
                    else:
                        sense = tagged_sents[i][j].label()
                        sense_dict[untagged_sents[i][j]].append(sense)
            else:
                sense_dict[untagged_sents[i][j]].append('none')
    for key in sense_dict.keys():
        sense_dict[key] = max(set(sense_dict[key]), key = sense_dict[key].count)
    return sense_dict
    

In [11]:
def most_frequent_sense(word,sense_dict):
    if word in sense_dict.keys():
        return sense_dict[word]
    else:
        return 'none'

In [12]:
def predict(tagged_sents,untagged_sents,sense_dict):
    actual = []
    pred = []
    count = 0
    for i in range(len(tagged_sents)):
        sentence = []
        if len(tagged_sents[i])<=1:
            continue
        for j in range(len(tagged_sents[i])):
            
            word = ''
        
            if isinstance(tagged_sents[i][j],nltk.Tree):
                try:
                    if tagged_sents[i][j].height() == 3:
                        for tree in tagged_sents[i][j]:
                            if(tree.label() == 'NE'):
                                
                                word = "_".join(tree.leaves())
                                actual.append('NE')
                                
                    else:
                        
                        word = "_".join(tagged_sents[i][j].leaves())
                        sense = tagged_sents[i][j].label().synset().name()
                        actual.append(sense)
                except:
                    if (tagged_sents[i][j].label()=='NE'):
                        word = "_".join(tagged_sents[i][j].leaves())
                        actual.append('NE')
                    else:
                        sense = tagged_sents[i][j].label()
                        word = "_".join(tagged_sents[i][j].leaves())
                        actual.append(sense)
            else:
                actual.append('none')
                word = "_".join(tagged_sents[i][j])
            pred.append(most_frequent_sense(word,sense_dict))
        if count%500 == 0:
            print("Steps Completed ",count)
        count += 1
        
    return actual,pred

In [13]:
semcor_tagged_sents = semcor.tagged_sents(tag='sem')
semcor_untagged_sents = semcor.sents()
sense_dict = get_most_frequent_sense_dict(semcor_tagged_sents,semcor_untagged_sents)

In [14]:
actual,pred = predict(semcor_tagged_sents,semcor_untagged_sents,sense_dict)

Steps Completed  0
Steps Completed  500
Steps Completed  1000
Steps Completed  1500
Steps Completed  2000
Steps Completed  2500
Steps Completed  3000
Steps Completed  3500
Steps Completed  4000
Steps Completed  4500
Steps Completed  5000
Steps Completed  5500
Steps Completed  6000
Steps Completed  6500
Steps Completed  7000
Steps Completed  7500
Steps Completed  8000
Steps Completed  8500
Steps Completed  9000
Steps Completed  9500
Steps Completed  10000
Steps Completed  10500
Steps Completed  11000
Steps Completed  11500
Steps Completed  12000
Steps Completed  12500
Steps Completed  13000
Steps Completed  13500
Steps Completed  14000
Steps Completed  14500
Steps Completed  15000
Steps Completed  15500
Steps Completed  16000
Steps Completed  16500
Steps Completed  17000
Steps Completed  17500
Steps Completed  18000
Steps Completed  18500
Steps Completed  19000
Steps Completed  19500
Steps Completed  20000
Steps Completed  20500
Steps Completed  21000
Steps Completed  21500
Steps Comple

In [15]:

y_true = [senses[i] for i in actual]
y_pred = [senses.get(i, senses['none']) for i in pred]

In [16]:
import sklearn.metrics as sm
print(sm.accuracy_score(y_true, y_pred))
print(sm.f1_score(y_true, y_pred, average='weighted'))
print(sm.precision_score(y_true, y_pred,average='weighted'))
print(sm.recall_score(y_true, y_pred,average='weighted'))

0.7515550766203558
0.6644547875469438
0.6323030066934414
0.7515550766203558
